# Xgboost

Ref: https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer

In [2]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [3]:
def feature_inputer(
    df:pd.DataFrame,
    to_mode = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],
    to_median = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
) -> pd.DataFrame:
    for col in to_mode:
        mode_inputer = SimpleImputer(strategy='most_frequent')
        df[col] = mode_inputer.fit_transform(df[[col]])

    
    for col in to_median:
        inputer = SimpleImputer(strategy='median')
        df[col] = inputer.fit_transform(df[[col]])
    return df

def dtype_memory_reducer(df: pd.DataFrame) -> pd.DataFrame:
    df['CryoSleep'] = df['CryoSleep'].astype(bool)
    df['VIP'] = df['VIP'].astype(bool)

    df['HomePlanet'] = df['HomePlanet'].astype('category')
    df['Destination'] = df['Destination'].astype('category')
    return df

def outliers_to_log(df:pd.DataFrame) -> pd.DataFrame:
    to_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in to_log:
        df[col] = np.log(df[col] + 1)
    return df

def cabin_inputer(df:pd.DataFrame) -> pd.DataFrame:
    cabin_features = df['Cabin'].str.split("/",expand=True)[[0,2]].rename(columns={0:'Deck',2:'side'})
    df = pd.concat([df,cabin_features],axis=1)
    df['Deck'] = df['Deck'].fillna('G').replace("T","G")
    df = df.drop(['side','Cabin'],axis=1)
    return df

def vip_knn_input(df: pd.DataFrame) -> pd.DataFrame:
    inputer = KNNImputer(n_neighbors=5)
    df['VIP'] = inputer.fit_transform(df[['VIP','RoomService']])[:,0]
    return df

def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = cabin_inputer(df)
    df = feature_inputer(df)
    df = vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = outliers_to_log(df)
    df = dtype_memory_reducer(df)
    return df

def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

In [4]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)

In [5]:
df_train.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,,
0001_01,False,39.0,False,0.00000,0.000000,0.000000,0.000000,0.000000,False,True,1,0,0,1,1,0,0,0,0,0
0002_01,False,24.0,False,4.70048,2.302585,3.258097,6.309918,3.806662,True,False,0,0,0,1,0,0,0,0,1,0


In [6]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [7]:
import xgboost as xgb
from sklearn.model_selection import KFold, GridSearchCV

## Tuning

In [12]:
params = {
 'max_depth': [4, 5, 8, 10],
 'learning_rate': [0.01,0.1,1],
 'n_estimators': [10, 100, 200],
 'subsample':[0.6, 1],
 'colsample_bytree': [0.3, 0.7,1],
}

In [13]:
grid = GridSearchCV(
    xgb.XGBClassifier(),
    param_grid=params,
    scoring='accuracy',
    cv=KFold(5, shuffle=True, random_state=42),
    verbose=3,
    return_train_score=True
)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=4, n_estimators=10, subsample=0.6;, score=(train=0.773, test=0.772) total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=4, n_estimators=10, subsample=0.6;, score=(train=0.774, test=0.754) total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=4, n_estimators=10, subsample=0.6;, score=(train=0.774, test=0.767) total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=4, n_estimators=10, subsample=0.6;, score=(train=0.765, test=0.764) total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=4, n_estimators=10, subsample=0.6;, score=(train=0.761, test=0.766) total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=4, n_estimators=10, subsample=1;, score=(train=0.762, test=0.752) total time=   0.0s
[CV 2/5] END colsample_byt

[CV 5/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=0.6;, score=(train=0.785, test=0.791) total time=   0.3s
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=1;, score=(train=0.788, test=0.775) total time=   0.2s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=1;, score=(train=0.786, test=0.765) total time=   0.2s
[CV 3/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=1;, score=(train=0.788, test=0.772) total time=   0.3s
[CV 4/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=1;, score=(train=0.786, test=0.778) total time=   0.3s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=1;, score=(train=0.787, test=0.789) total time=   0.3s
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=8, n_estimators=10, subsamp

[CV 4/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=10, n_estimators=100, subsample=1;, score=(train=0.811, test=0.773) total time=   0.2s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=10, n_estimators=100, subsample=1;, score=(train=0.812, test=0.781) total time=   0.2s
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=10, n_estimators=200, subsample=0.6;, score=(train=0.819, test=0.779) total time=   0.6s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=10, n_estimators=200, subsample=0.6;, score=(train=0.812, test=0.764) total time=   0.5s
[CV 3/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=10, n_estimators=200, subsample=0.6;, score=(train=0.812, test=0.776) total time=   0.5s
[CV 4/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=10, n_estimators=200, subsample=0.6;, score=(train=0.818, test=0.778) total time=   0.5s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.01, max_depth=10, n_estimator

[CV 4/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.826, test=0.795) total time=   0.1s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.827, test=0.801) total time=   0.1s
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1;, score=(train=0.830, test=0.794) total time=   0.1s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1;, score=(train=0.829, test=0.795) total time=   0.1s
[CV 3/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1;, score=(train=0.829, test=0.801) total time=   0.1s
[CV 4/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1;, score=(train=0.828, test=0.798) total time=   0.1s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1

[CV 4/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.798, test=0.768) total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.804, test=0.772) total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.874, test=0.790) total time=   0.2s
[CV 2/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.872, test=0.790) total time=   0.3s
[CV 3/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.871, test=0.788) total time=   0.5s
[CV 4/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.875, test=0.786) total time=   0.5s
[CV 5/5] END colsample_bytree=0.3, learning_rate=0.1, max_depth=10, n_estimators=100, su

...

[CV 3/5] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.838, test=0.805) total time=   0.3s
[CV 4/5] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.837, test=0.804) total time=   0.2s
[CV 5/5] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.836, test=0.805) total time=   0.2s
[CV 1/5] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1;, score=(train=0.827, test=0.798) total time=   0.2s
[CV 2/5] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1;, score=(train=0.834, test=0.803) total time=   0.2s
[CV 3/5] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1;, score=(train=0.832, test=0.808) total time=   0.2s
[CV 4/5] END colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=1;, score=(tr

[CV 4/5] END colsample_bytree=1, learning_rate=0.1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.849, test=0.791) total time=   0.0s
[CV 5/5] END colsample_bytree=1, learning_rate=0.1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.847, test=0.795) total time=   0.0s
[CV 1/5] END colsample_bytree=1, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.905, test=0.787) total time=   0.5s
[CV 2/5] END colsample_bytree=1, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.906, test=0.784) total time=   0.5s
[CV 3/5] END colsample_bytree=1, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.907, test=0.790) total time=   0.5s
[CV 4/5] END colsample_bytree=1, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.903, test=0.796) total time=   0.5s
[CV 5/5] END colsample_bytree=1, learning_rate=0.1, max_depth=10, n_estimators=100, subsample=0.6;, 

[CV 3/5] END colsample_bytree=1, learning_rate=1, max_depth=5, n_estimators=10, subsample=1;, score=(train=0.831, test=0.796) total time=   0.0s
[CV 4/5] END colsample_bytree=1, learning_rate=1, max_depth=5, n_estimators=10, subsample=1;, score=(train=0.830, test=0.795) total time=   0.0s
[CV 5/5] END colsample_bytree=1, learning_rate=1, max_depth=5, n_estimators=10, subsample=1;, score=(train=0.834, test=0.795) total time=   0.0s
[CV 1/5] END colsample_bytree=1, learning_rate=1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.908, test=0.760) total time=   0.2s
[CV 2/5] END colsample_bytree=1, learning_rate=1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.908, test=0.759) total time=   0.2s
[CV 3/5] END colsample_bytree=1, learning_rate=1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.910, test=0.754) total time=   0.2s
[CV 4/5] END colsample_bytree=1, learning_rate=1, max_depth=5, n_estimators=100, subsample=0.6;, score=(train=0.911, test

[CV 1/5] END colsample_bytree=1, learning_rate=1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.910, test=0.769) total time=   0.0s
[CV 2/5] END colsample_bytree=1, learning_rate=1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.911, test=0.772) total time=   0.0s
[CV 3/5] END colsample_bytree=1, learning_rate=1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.895, test=0.786) total time=   0.0s
[CV 4/5] END colsample_bytree=1, learning_rate=1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.911, test=0.773) total time=   0.0s
[CV 5/5] END colsample_bytree=1, learning_rate=1, max_depth=10, n_estimators=10, subsample=1;, score=(train=0.897, test=0.770) total time=   0.0s
[CV 1/5] END colsample_bytree=1, learning_rate=1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.935, test=0.757) total time=   0.5s
[CV 2/5] END colsample_bytree=1, learning_rate=1, max_depth=10, n_estimators=100, subsample=0.6;, score=(train=0.934, tes

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     int...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.3, 0.7, 1],
                         'learning_rate': [0.01, 0.1, 1],
                         'max_depth': [4, 5, 8, 10],
                         'n_estimators': [10, 100, 200],
                         'subsample': [0.6, 1]},
             return_train_score=True, scoring='accuracy', verbose=3)

In [14]:
grid.best_params_

{'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 200,
 'subsample': 1}

In [15]:
pd.DataFrame(grid.cv_results_)[['mean_train_score','std_train_score','mean_test_score','std_test_score']].loc[grid.best_index_]

mean_train_score    0.850742
std_train_score     0.001702
mean_test_score     0.802830
std_test_score      0.005101
Name: 179, dtype: float64

In [17]:
model = grid.best_estimator_
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=5, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=200, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

## Cross Validation

# Submission

In [16]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,
0013_01,True,27.0,False,0.0,0.000000,0.0,0.00000,0.0,True,0,0,0,1,0,0,0,0,0,1
0018_01,False,19.0,False,0.0,2.302585,0.0,7.94591,0.0,False,0,0,0,1,0,0,0,0,1,0


In [18]:
y_test = model.predict(X_test)

In [20]:
y_test

array([1, 0, 1, ..., 1, 1, 1])

In [21]:
submission = pd.Series(y_test, index=[X_test.index]).astype(bool).to_frame('Transported')
submission.to_csv('data/submission_xgboost_2022-11-04.csv')